# FindCare Backend (Gradio + FastAPI) — Implementation Notebook

This notebook implements the **6 required browser→server APIs** for the FindCare React frontend:

- `/api/prompt` (multipart/form-data)
- `/api/graphic-content` (JSON)
- `/api/scrollable-output` (JSON)
- `/api/session-summary` (JSON)
- `/api/header` (JSON)
- `/api/button-manager` (JSON)

**Critical demo requirement**: On initial load, `/api/graphic-content` must return an **interactive US map with all 50 states clickable**.


## 0) Install dependencies (run once)
Skip if already installed.

In [32]:
# Install dependencies (run once)
%pip install -U fastapi "uvicorn[standard]" gradio python-multipart pydantic pillow PyPDF2 pytesseract plotly



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1) Imports + configuration

In [17]:
# =============================================================================
# File: FindCareInterfaceCatalogue.ipynb (cells)
# Author: Skip Snow
# Co-Author: GPT-5
# Copyright (c) 2025 Skip Snow. All rights reserved.
# =============================================================================

from __future__ import annotations

import os, re, json, uuid, time, html, logging
from dataclasses import dataclass, asdict
from datetime import datetime, timezone
from typing import Any, Dict, List, Optional, Tuple

from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware

# Optional imports (best-effort)
try:
    from PIL import Image
except Exception:
    Image = None

try:
    import pytesseract
except Exception:
    pytesseract = None

try:
    import PyPDF2
except Exception:
    PyPDF2 = None

try:
    import gradio as gr
except Exception:
    gr = None

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("findcare-backend")

HOST = os.getenv("FINDCARE_HOST", "127.0.0.1")
PORT = int(os.getenv("FINDCARE_PORT", "7860"))

ALLOWED_ORIGINS = [
    "http://localhost:5173",
    "http://127.0.0.1:5173",
    "http://localhost:3000",
    "http://127.0.0.1:3000",
]

DEFAULT_SUMMARY_INTERVAL_SEC = int(os.getenv("FINDCARE_SUMMARY_INTERVAL_SEC", "60"))
APP_TONE = "austere"

def utc_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def new_id(prefix: str) -> str:
    return f"{prefix}-{uuid.uuid4().hex[:12]}"


## 2) Mock provider data (replace with your real provider dataset)

In [18]:
MOCK_PROVIDERS: List[Dict[str, Any]] = [
    {
        "id": "prov-0001",
        "name": "Stanford Health Care – Cardiology",
        "specialty": "Cardiology",
        "state": "CA",
        "city": "Palo Alto",
        "distance": "2.3 miles",
        "rating": 4.8,
        "accepts_insurance": ["Anthem", "Blue Cross", "Kaiser"],
    },
    {
        "id": "prov-0002",
        "name": "UCLA Medical Center – Cardiology",
        "specialty": "Cardiology",
        "state": "CA",
        "city": "Los Angeles",
        "distance": "5.1 miles",
        "rating": 4.9,
        "accepts_insurance": ["Anthem", "Blue Cross", "UCLA Health"],
    },
    {
        "id": "prov-0003",
        "name": "Mayo Clinic – Primary Care",
        "specialty": "Primary Care",
        "state": "MN",
        "city": "Rochester",
        "distance": "—",
        "rating": 4.7,
        "accepts_insurance": ["Blue Cross", "Aetna"],
    },
    {
        "id": "prov-0004",
        "name": "Mass General – Endocrinology",
        "specialty": "Endocrinology",
        "state": "MA",
        "city": "Boston",
        "distance": "—",
        "rating": 4.6,
        "accepts_insurance": ["Anthem", "Harvard Pilgrim"],
    },
]

def search_providers(state: Optional[str]=None, specialty: Optional[str]=None, insurance: Optional[str]=None, limit: int=50) -> List[Dict[str, Any]]:
    results = MOCK_PROVIDERS
    if state:
        results = [p for p in results if p.get("state") == state.upper()]
    if specialty:
        s = specialty.strip().lower()
        results = [p for p in results if s in str(p.get("specialty","")).lower()]
    if insurance:
        ins = insurance.strip()
        results = [p for p in results if ins in (p.get("accepts_insurance") or [])]
    return results[:max(1,int(limit))]


## 3) US states map data (50 clickable regions)

In [19]:
US_STATES: List[Tuple[str, str]] = [
    ("AL","Alabama"),("AK","Alaska"),("AZ","Arizona"),("AR","Arkansas"),("CA","California"),
    ("CO","Colorado"),("CT","Connecticut"),("DE","Delaware"),("FL","Florida"),("GA","Georgia"),
    ("HI","Hawaii"),("ID","Idaho"),("IL","Illinois"),("IN","Indiana"),("IA","Iowa"),
    ("KS","Kansas"),("KY","Kentucky"),("LA","Louisiana"),("ME","Maine"),("MD","Maryland"),
    ("MA","Massachusetts"),("MI","Michigan"),("MN","Minnesota"),("MS","Mississippi"),("MO","Missouri"),
    ("MT","Montana"),("NE","Nebraska"),("NV","Nevada"),("NH","New Hampshire"),("NJ","New Jersey"),
    ("NM","New Mexico"),("NY","New York"),("NC","North Carolina"),("ND","North Dakota"),("OH","Ohio"),
    ("OK","Oklahoma"),("OR","Oregon"),("PA","Pennsylvania"),("RI","Rhode Island"),("SC","South Carolina"),
    ("SD","South Dakota"),("TN","Tennessee"),("TX","Texas"),("UT","Utah"),("VT","Vermont"),
    ("VA","Virginia"),("WA","Washington"),("WV","West Virginia"),("WI","Wisconsin"),("WY","Wyoming"),
]

def provider_count_by_state(state_code: str) -> int:
    return sum(1 for p in MOCK_PROVIDERS if p.get("state") == state_code)

def build_us_states_map(selected: Optional[str]=None) -> Dict[str, Any]:
    regions = []
    for code, name in US_STATES:
        regions.append({
            "id": code,
            "name": name,
            "displayName": name,
            "tooltip": f"Click to view {name} providers",
            "color": "#3b82f6" if selected and selected.upper()==code else "#94a3b8",
            "data": {"providerCount": provider_count_by_state(code)}
        })
    return {
        "contentType": "map",
        "mapData": {
            "type": "us-states",
            "regions": regions,
            "interactionMode": "select-state",
            "selectedRegion": selected.upper() if selected else None
        }
    }


## 4) Sanitization + file extraction helpers (best-effort OCR/PDF)

In [20]:
def sanitize_html_allow_basic(markup: str) -> str:
    if markup is None:
        return ""
    escaped = html.escape(markup)
    allowed = ["b","strong","i","em","br","p","ul","ol","li","code","pre"]
    for tag in allowed:
        escaped = escaped.replace(f"&lt;{tag}&gt;", f"<{tag}>")
        escaped = escaped.replace(f"&lt;/{tag}&gt;", f"</{tag}>")
    return escaped

async def extract_text_from_upload(file: UploadFile) -> str:
    filename = file.filename or "uploaded_file"
    ctype = (file.content_type or "").lower()
    try:
        raw = await file.read()
    except Exception as e:
        return f"[{filename}] (error reading file: {e})"

    if ctype.startswith("image/") and Image is not None and pytesseract is not None:
        try:
            from io import BytesIO
            img = Image.open(BytesIO(raw))
            text = (pytesseract.image_to_string(img) or "").strip()
            return f"[{filename} OCR]\n{text}\n" if text else f"[{filename}] (OCR produced no text)"
        except Exception as e:
            return f"[{filename}] (OCR failed: {e})"

    if ctype == "application/pdf" and PyPDF2 is not None:
        try:
            from io import BytesIO
            reader = PyPDF2.PdfReader(BytesIO(raw))
            pages = [(p.extract_text() or "") for p in reader.pages]
            text = "\n".join(pages).strip()
            return f"[{filename} PDF]\n{text}\n" if text else f"[{filename}] (PDF extraction produced no text)"
        except Exception as e:
            return f"[{filename}] (PDF extraction failed: {e})"

    return f"[{filename}] (uploaded, contentType={ctype}, size={len(raw)} bytes)"


## 5) In-memory session store (replace with Redis/Mongo in production)

In [21]:
@dataclass
class ChatMessage:
    id: str
    role: str
    content: str
    timestamp: str

@dataclass
class SessionState:
    sessionId: str
    createdAt: str
    selectedState: Optional[str] = None
    messages: List[ChatMessage] = None
    summary: str = ""
    lastSummaryAt: Optional[str] = None

SESSIONS: Dict[str, SessionState] = {}

def get_or_create_session(session_id: Optional[str]) -> SessionState:
    sid = (session_id or "").strip() or new_id("session")
    if sid not in SESSIONS:
        SESSIONS[sid] = SessionState(sessionId=sid, createdAt=utc_iso(), selectedState=None, messages=[])
    return SESSIONS[sid]

def append_message(session: SessionState, role: str, content: str) -> ChatMessage:
    msg = ChatMessage(id=new_id("msg"), role=role, content=content, timestamp=utc_iso())
    session.messages.append(msg)
    return msg

def to_history_payload(session: SessionState, limit: Optional[int]=None, offset: int=0) -> Dict[str, Any]:
    msgs = session.messages[offset:]
    if limit is not None:
        msgs = msgs[:int(limit)]
    return {"messages":[asdict(m) for m in msgs], "total": len(session.messages), "hasMore": False}


## 6) FastAPI app + CORS + optional Gradio mount

In [22]:
app = FastAPI(title="FindCare Gradio Backend (FastAPI API Layer)", version="1.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=ALLOWED_ORIGINS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

if gr is not None:
    with gr.Blocks() as demo:
        gr.Markdown("### FindCare Backend\nThis Gradio UI is for local smoke-testing only.")
        gr.Markdown(f"- Host: `{HOST}`\n- Port: `{PORT}`\n- Time: `{utc_iso()}`")
    app = gr.mount_gradio_app(app, demo, path="/gradio")


new /gradio


2025-12-27 16:00:00,462 | INFO | HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


## 7) API: `/api/header`

In [23]:
@app.post("/api/header")
async def header_api(payload: Dict[str, Any]) -> JSONResponse:
    link = (payload or {}).get("link")
    if link not in {"secret-sause","about","contact","privacy-policy"}:
        return JSONResponse(status_code=400, content={"status":"error","message":"Invalid link"})

    if link == "contact":
        return JSONResponse(content={
            "type": "contact-info",
            "contactName": "Skip Snow",
            "contactEmail": "skip.snow@gmail.com",
            "mailtoSubject": "FindCare Inquiry",
            "fallbackMessage": "Please email Skip Snow at skip.snow@gmail.com"
        })

    if link == "secret-sause":
        content = sanitize_html_allow_basic(
            "<p><strong>Secret Sause</strong></p>"
            "<p>Tools/processes used to fuel FindCare: Gradio + FastAPI backend, provider/specialty data, and LLM orchestration.</p>"
        )
        return JSONResponse(content={"type":"page-content","content":content})

    if link == "about":
        content = sanitize_html_allow_basic(
            "<p><strong>About FindCare</strong></p>"
            "<p>FindCare helps users ask domain-specific questions and locate providers using existing provider & specialty data.</p>"
        )
        return JSONResponse(content={"type":"page-content","content":content})

    # privacy-policy
    content = sanitize_html_allow_basic(
        "<p><strong>Privacy Policy (MVP)</strong></p>"
        "<p>FindCare does not store passwords. Identified PHI may be accessed only with explicit user action and is not retained by default.</p>"
    )
    return JSONResponse(content={"type":"page-content","content":content})


## 8) API: `/api/session-summary` (wipe & replace)

In [24]:
def simple_summary(messages: List[ChatMessage], max_chars: int=800) -> str:
    if not messages:
        return "No conversation yet."
    last_user = next((m for m in reversed(messages) if m.role=="user"), None)
    last_assistant = next((m for m in reversed(messages) if m.role=="assistant"), None)
    parts = [f"Messages: {len(messages)}"]
    if last_user:
        parts.append(f"Latest question: {last_user.content[:200]}")
    if last_assistant:
        parts.append(f"Latest answer: {last_assistant.content[:200]}")
    out = " | ".join(parts)
    return out[:max_chars]

@app.post("/api/session-summary")
async def session_summary_api(payload: Dict[str, Any]) -> JSONResponse:
    action = (payload or {}).get("action")
    sessionId = (payload or {}).get("sessionId")
    session = get_or_create_session(sessionId)

    if action == "get-summary":
        session.summary = simple_summary(session.messages)
        session.lastSummaryAt = utc_iso()
        return JSONResponse(content={"summary": session.summary, "timestamp": session.lastSummaryAt, "nextUpdateIn": DEFAULT_SUMMARY_INTERVAL_SEC})

    if action == "copy-summary":
        logger.info("Summary copied | session=%s | method=%s | ts=%s", session.sessionId, payload.get("copyMethod"), payload.get("timestamp"))
        return JSONResponse(content={"logged": True})

    if action == "report-error":
        logger.warning("Summary overflow | session=%s | payload=%s", session.sessionId, payload)
        return JSONResponse(content={"acknowledged": True, "fallbackAction": "truncate", "fallbackContent": (session.summary or "")[:400]})

    return JSONResponse(status_code=400, content={"status":"error","message":"Invalid action"})


## 9) API: `/api/button-manager` (forms + stub integrations; never store passwords)

In [25]:
@app.post("/api/button-manager")
async def button_manager_api(payload: Dict[str, Any]) -> JSONResponse:
    action = (payload or {}).get("action")
    step = (payload or {}).get("step")
    sessionId = (payload or {}).get("sessionId")
    _ = get_or_create_session(sessionId)

    if action not in {"insurance-portal","emr-access"}:
        return JSONResponse(status_code=400, content={"status":"error","message":"Invalid action"})

    if step == "request-form":
        title = "Insurance Portal Access" if action=="insurance-portal" else "EMR Access"
        return JSONResponse(content={
            "formType": "popup",
            "formTitle": title,
            "formContent": {
                "agreement": {"text":"We will access your medical data but not retain identified information.","checkboxLabel":"I agree to the terms above","required": True},
                "fields": [
                    {"name":"username","label":"Portal Username","type":"text","required": True,"placeholder":"username"},
                    {"name":"password","label":"Portal Password","type":"password","required": True,"placeholder":"password"},
                    {"name":"portalUrl","label":"Portal URL or App Name","type":"url","required": True,"placeholder":"https://... or 'Epic MyChart'"},
                ],
                "submitButton":"Connect",
                "cancelButton":"Cancel"
            }
        })

    if step == "submit-credentials":
        creds = (payload or {}).get("credentials") or {}
        if not creds.get("agreementAccepted", False):
            return JSONResponse(status_code=400, content={"status":"error","errorCode":"CONNECTION_FAILED","message":"Agreement must be accepted to proceed.","retryAllowed": True})

        portal = creds.get("portalUrl","")
        username = creds.get("username","")

        if action=="insurance-portal":
            supported = ["Kaiser","UCLA","Anthem"]
            if not any(s.lower() in portal.lower() for s in supported):
                return JSONResponse(status_code=400, content={"status":"error","errorCode":"UNSUPPORTED_PORTAL","message":"Unsupported insurance portal for V1.","retryAllowed": False,"supportedPortals": supported})

        if action=="emr-access":
            if "epic" not in portal.lower():
                return JSONResponse(status_code=400, content={"status":"error","errorCode":"UNSUPPORTED_PORTAL","message":"Unsupported EMR for V1. Only Epic is supported.","retryAllowed": False,"supportedPortals": ["Epic"]})

        return JSONResponse(content={"status":"success","message": f"Connected (stub) as {username or '[user]'}","dataRetrieved":{"summary":"Retrieved sample de-identified context (stub).","recordCount":3,"lastUpdated": utc_iso()}})

    return JSONResponse(status_code=400, content={"status":"error","message":"Invalid step"})


## 10) API: `/api/scrollable-output`

In [26]:
@app.post("/api/scrollable-output")
async def scrollable_output_api(payload: Dict[str, Any]) -> JSONResponse:
    action = (payload or {}).get("action")
    sessionId = (payload or {}).get("sessionId")
    session = get_or_create_session(sessionId)

    if action == "append":
        message = (payload or {}).get("message") or {}
        role = message.get("role")
        content = message.get("content","")
        if role not in {"user","assistant"}:
            return JSONResponse(status_code=400, content={"status":"error","message":"Invalid role"})
        msg = append_message(session, role, content)
        return JSONResponse(content={"success": True, "messageId": msg.id, "timestamp": msg.timestamp})

    if action == "get-history":
        limit = (payload or {}).get("limit")
        offset = int((payload or {}).get("offset") or 0)
        return JSONResponse(content=to_history_payload(session, limit=limit, offset=offset))

    if action == "log-copy":
        logger.info("Copy event | session=%s | messageIds=%s | type=%s | ts=%s", session.sessionId, payload.get("messageIds"), payload.get("copyType"), payload.get("timestamp"))
        return JSONResponse(content={"logged": True})

    return JSONResponse(status_code=400, content={"status":"error","message":"Invalid action"})


## 11) API: `/api/graphic-content` (map/table/chart)

In [27]:
def build_provider_table(providers: List[Dict[str, Any]], page: int=1, page_size: int=25) -> Dict[str, Any]:
    page = max(1,int(page))
    page_size = max(1,int(page_size))
    start = (page-1)*page_size
    subset = providers[start:start+page_size]

    headers = [
        {"key":"name","label":"Provider Name","sortable": True,"editable": False},
        {"key":"specialty","label":"Specialty","sortable": True,"editable": False},
        {"key":"city","label":"City","sortable": True,"editable": False},
        {"key":"state","label":"State","sortable": True,"editable": False},
        {"key":"rating","label":"Rating","sortable": True,"editable": False},
        {"key":"notes","label":"Notes","sortable": False,"editable": True},
    ]
    rows = []
    for p in subset:
        rows.append({"id": p["id"], "name": p.get("name",""), "specialty": p.get("specialty",""), "city": p.get("city",""), "state": p.get("state",""), "rating": p.get("rating",""), "notes": p.get("notes","")})
    return {"contentType":"table","tableData":{"headers": headers,"rows": rows,"totalRows": len(providers),"page": page,"pageSize": page_size,"editableColumns":["notes"]}}

@app.post("/api/graphic-content")
async def graphic_content_api(payload: Dict[str, Any]) -> JSONResponse:
    action = (payload or {}).get("action")
    sessionId = (payload or {}).get("sessionId")
    session = get_or_create_session(sessionId)

    if action == "get-content":
        ctx = (payload or {}).get("context") or {}
        selected = ctx.get("selectedState") or session.selectedState
        return JSONResponse(content=build_us_states_map(selected=selected))

    if action == "map-click":
        regionId = (payload or {}).get("regionId")
        if not regionId:
            return JSONResponse(status_code=400, content={"status":"error","message":"regionId required"})
        session.selectedState = str(regionId).upper()
        providers = search_providers(state=session.selectedState)
        return JSONResponse(content=build_provider_table(providers))

    if action == "edit-cell":
        rowId = (payload or {}).get("rowId")
        columnKey = (payload or {}).get("columnKey")
        newValue = (payload or {}).get("newValue","")
        if columnKey != "notes":
            return JSONResponse(status_code=400, content={"status":"error","message":"Only 'notes' editable in MVP"})
        updated = False
        for p in MOCK_PROVIDERS:
            if p["id"] == rowId:
                p["notes"] = str(newValue)
                updated = True
                break
        return JSONResponse(content={"success": True, "updated": updated})

    if action == "chart-drill-down":
        state = session.selectedState
        providers = search_providers(state=state) if state else MOCK_PROVIDERS
        counts: Dict[str,int] = {}
        for p in providers:
            counts[p.get("specialty","Unknown")] = counts.get(p.get("specialty","Unknown"), 0) + 1
        chart_data = []
        for i,(label,value) in enumerate(sorted(counts.items(), key=lambda kv: kv[1], reverse=True)):
            chart_data.append({"id": f"bar-{i}", "label": label, "value": value, "color":"#60a5fa", "drillDownAvailable": False})
        return JSONResponse(content={"contentType":"chart","chartData":{"chartType":"bar","title": f"Providers by Specialty{(' in '+state) if state else ''}","xAxisLabel":"Specialty","yAxisLabel":"Count","data": chart_data,"interactive": True}})

    return JSONResponse(status_code=400, content={"status":"error","message":"Invalid action"})


## 12) API: `/api/prompt` (multipart/form-data)

In [28]:
def generate_assistant_reply(session: SessionState, prompt: str, file_context: str) -> str:
    prompt_clean = (prompt or "").strip()
    if not prompt_clean:
        return "Please enter a question about providers, specialties, locations, or insurance context."

    guidance = "I can help with providers, specialties, locations, and (optionally) insurance/EMR context. "
    state = session.selectedState
    state_note = f"Current state filter: {state}. " if state else "No state selected yet. You can click a state on the map. "

    specialty = None
    low = prompt_clean.lower()
    if "cardio" in low:
        specialty = "Cardiology"
    elif "endocr" in low:
        specialty = "Endocrinology"
    elif "primary" in low:
        specialty = "Primary Care"

    results = search_providers(state=state, specialty=specialty) if (state or specialty) else []
    if results:
        lines = [f"- {p['name']} ({p['city']}, {p['state']}) — {p['specialty']} (rating {p['rating']})" for p in results]
        return guidance + state_note + "Here are matching providers:\n" + "\n".join(lines)

    extra = "\n\n(Uploaded file context detected; incorporate as de-identified context in V1.)" if file_context else ""
    return guidance + state_note + f"You asked: {prompt_clean}{extra}"

@app.post("/api/prompt")
async def prompt_api(
    prompt: str = Form(...),
    sessionId: str = Form(None),
    timestamp: str = Form(None),
    files: List[UploadFile] = File(default=[])
) -> JSONResponse:
    session = get_or_create_session(sessionId)
    append_message(session, "user", prompt)

    file_texts = []
    for f in files or []:
        file_texts.append(await extract_text_from_upload(f))
    file_context = "\n\n".join([t for t in file_texts if t])

    assistant_text = generate_assistant_reply(session, prompt, file_context)
    assistant_msg = append_message(session, "assistant", assistant_text)

    return JSONResponse(content={
        "success": True,
        "messageId": assistant_msg.id,
        "response": {"content": assistant_text, "contentType": "markdown"},
        "timestamp": assistant_msg.timestamp,
        "sessionId": session.sessionId
    })


## 13) Health + minimal landing page

In [29]:
@app.get("/health")
async def health() -> Dict[str, Any]:
    return {"status":"ok","time": utc_iso(),"tone": APP_TONE}

@app.get("/")
async def root() -> HTMLResponse:
    html_doc = (
        "<html><head><title>FindCare Backend</title></head><body>"
        "<h3>FindCare Backend is running</h3>"
        "<ul>"
        "<li>Health: <a href='/health'>/health</a></li>"
        "<li>Optional Gradio UI: <a href='/gradio'>/gradio</a></li>"
        "</ul>"
        "<p>React frontend should call: /api/prompt, /api/graphic-content, /api/scrollable-output, /api/session-summary, /api/header, /api/button-manager</p>"
        "</body></html>"
    )
    return HTMLResponse(content=html_doc)


## 14) Start server + open browser

In [30]:
import threading, webbrowser

def run_server():
    import uvicorn
    uvicorn.run(app, host=HOST, port=PORT, log_level="info")

_server_thread = None

def start_backend(open_path: str="/"):
    global _server_thread
    if _server_thread and _server_thread.is_alive():
        print(f"Server already running on http://{HOST}:{PORT}{open_path}")
        return
    _server_thread = threading.Thread(target=run_server, daemon=True)
    _server_thread.start()
    time.sleep(1.0)
    url = f"http://{HOST}:{PORT}{open_path}"
    print(f"Opening: {url}")
    try:
        webbrowser.open(url, new=1)
    except Exception as e:
        print(f"Could not open browser automatically: {e}\nOpen this URL manually: {url}")

start_backend("/")


INFO:     Started server process [44932]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 7860): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Opening: http://127.0.0.1:7860/
